In [1]:
from model_functions import analyze_token_sequence, predict, write_midi, get_token_flags
import json
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from transformers import GPT2Config, GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling, TrainingArguments, Trainer

PATH_VOCAB = "../0_data/5_vocabs"
PATH_MODELS = "../0_data/7_models"
PATH_MODELS_CONFIG = "../0_data/7_models/config"
PATH_PRED = "../0_data/8_predictions"
PATH_TOKENS = "../0_data/8_predictions/tokens"
PATH_MIDI = "../0_data/8_predictions/midi"

for path in [PATH_PRED, PATH_TOKENS, PATH_MIDI]:
    if not os.path.exists(path):
        os.makedirs(path)

In [2]:
vocab_configs = {
    "a1" : {
        "pitch_range": 128,
        "duration_steps": 64,
        "triole_tokens": False,
    },
    "a2" : {
        "pitch_range": 128,
        "duration_steps": 64,
        "triole_tokens": True,
    },
    "a3" : {
        "pitch_range": 128,
        "duration_steps": 32,
        "triole_tokens": False,
    },
    "b" : {
        "pitch_range": 128,
        "duration_steps": 64,
        "triole_tokens": False,
    },
    "c" : {
        "pitch_range": 36,
        "duration_steps": 64,
        "triole_tokens": False,
    },
    "d" : {
        "pitch_range": 36,
        "duration_steps": 32,
        "triole_tokens": True,
    }
}

In [3]:
model_df = pd.read_excel(f"{PATH_MODELS}/comparison_model_stats.xlsx", index_col="Unnamed: 0")
model_df

,name,max_length,emb_dim,attention_heads,layers,dropout,learning_rate,epochs,batch_size,ran,runtime,runtime_min,min_loss,at_epoch,perplexity,incorrect_notes,correct_notes,correct_rate
0,a1_50,1024,256,4,6,0,0.001,50,4,yes,646.7671,10.78,1.346300,43,679.524536,0.42,322.90,1.0
1,a2_50,1024,256,4,6,0,0.001,50,4,yes,631.2694,10.52,1.177220,46,289.333679,0.36,281.15,1.0
2,a3_50,1024,256,4,6,0,0.001,50,4,yes,662.6886,11.04,1.390447,36,275.937103,0.40,321.41,1.0
3,b_50,1024,256,4,6,0,0.001,50,4,yes,652.5901,10.88,1.352880,36,442.333618,0.24,322.21,1.0
4,c_50,1024,256,4,6,0,0.001,50,4,yes,643.5783,10.73,1.369530,36,364.786407,0.32,321.04,1.0
5,d_50,1024,256,4,6,0,0.001,50,4,yes,634.2541,10.57,1.167735,50,486.180145,0.29,278.94,1.0
6,a1,1024,256,4,6,0,0.001,20,4,yes,253.2759,4.22,1.378125,20,94.867638,0.55,322.85,1.0
7,a2,1024,256,4,6,0,0.001,20,4,yes,262.6371,4.38,1.272294,20,103.435127,0.46,278.76,1.0
8,a3,1024,256,4,6,0,0.001,20,4,yes,263.4213,4.39,1.408199,20,202.802673,0.40,321.03,1.0
9,b,1024,256,4,6,0,0.001,20,4,yes,263.1353,4.39,1.351377,20,88.056297,0.58,321.76,1.0


In [4]:
TICKS_PER_BEAT = 1024
TICKS_PER_MIN_DURATION = TICKS_PER_BEAT*4/32
    
for index, row in model_df.iterrows():

    # only models that ran
    if row["ran"] == "no" or pd.notnull(row["incorrect_notes"]) or row["runtime"] == "too big - cuda error":
        continue
    
    model_name = row["name"]
    model_vocab = model_name.split("_")[0]
    print(model_name, model_vocab)
    if not os.path.exists(f"{PATH_MIDI}/{model_name}"):
        os.makedirs(f"{PATH_MIDI}/{model_name}")

        # get token flags and duration bins
    token_flags = get_token_flags(vocab_configs[model_vocab])
    duration_steps = vocab_configs[model_vocab]["duration_steps"]
    duration_bins = np.arange(TICKS_PER_MIN_DURATION, (TICKS_PER_MIN_DURATION*duration_steps)+1, TICKS_PER_MIN_DURATION, dtype=int)

    # create tokenizer
    tokenizer = GPT2Tokenizer(
        vocab_file=f"{PATH_VOCAB}/vocab_{model_vocab}.json", 
        merges_file=f"{PATH_VOCAB}/merges.txt")
    tokenizer.add_special_tokens({'pad_token': 'PAD', 'bos_token': 'BOS', 'eos_token': 'EOS',})
    
    # get vocabulary
    with open(f"{PATH_VOCAB}/vocab_{model_vocab}.json", "r") as fp:
        vocab = json.load(fp)
    token2word = {token: word for word, token in vocab.items()}

    # load model
    model = GPT2LMHeadModel.from_pretrained(f"{PATH_MODELS_CONFIG}/{model_name}/end_version")
    
    # make predictions save
    output = predict(model, tokenizer, samples=100, max_length=row["max_length"])
    data_generated = {"data": output}
    
    with open(f"{PATH_TOKENS}/{model_name}.json", "w") as fp:
        json.dump(data_generated, fp)
    
    # analyze tokens and save as midi_files
    correct_notes = 0
    incorrect_notes = 0
    for idx, pred in enumerate(output):
        an = analyze_token_sequence(pred, token_flags)
        correct_notes += an["start-pos-pitch-duration"] + an["start-pos-pitch-duration-dtriole"] + an["start-pos-ptriole-pitch-duration"] + an["start-pos-ptriole-pitch-duration-dtriole"]
        incorrect_notes += write_midi(output[idx], token2word, duration_bins, f"{PATH_MIDI}/{model_name}/generated_midi_{idx}.midi")
    
    model_df.at[index,"correct_notes"] = (correct_notes/100).__round__(2)
    model_df.at[index,"incorrect_notes"] = (incorrect_notes/100).__round__(2)
    model_df.at[index,"correct_rate"] = (correct_notes/(correct_notes+incorrect_notes)).__round__(2)
    
model_df

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:119 for open-end generation.


c c
midi saved in ../0_data/8_predictions/midi/c/generated_midi_0.midi
Number of incorrect notes: 1
midi saved in ../0_data/8_predictions/midi/c/generated_midi_1.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/c/generated_midi_2.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/c/generated_midi_3.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/c/generated_midi_4.midi
Number of incorrect notes: 1
midi saved in ../0_data/8_predictions/midi/c/generated_midi_5.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/c/generated_midi_6.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/c/generated_midi_7.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/c/generated_midi_8.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/c/generated_midi_9.midi
Number of incorrect notes: 2
midi saved in ../0_data/8_prediction

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:90 for open-end generation.


midi saved in ../0_data/8_predictions/midi/c/generated_midi_97.midi
Number of incorrect notes: 2
midi saved in ../0_data/8_predictions/midi/c/generated_midi_98.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/c/generated_midi_99.midi
Number of incorrect notes: 0
d d
midi saved in ../0_data/8_predictions/midi/d/generated_midi_0.midi
Number of incorrect notes: 1
midi saved in ../0_data/8_predictions/midi/d/generated_midi_1.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/d/generated_midi_2.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/d/generated_midi_3.midi
Number of incorrect notes: 1
midi saved in ../0_data/8_predictions/midi/d/generated_midi_4.midi
Number of incorrect notes: 0
midi saved in ../0_data/8_predictions/midi/d/generated_midi_5.midi
Number of incorrect notes: 1
midi saved in ../0_data/8_predictions/midi/d/generated_midi_6.midi
Number of incorrect notes: 1
midi saved in ../0_data/8_predict

,name,max_length,emb_dim,attention_heads,layers,dropout,learning_rate,epochs,batch_size,ran,runtime,runtime_min,min_loss,at_epoch,perplexity,incorrect_notes,correct_notes,correct_rate
0,a1_50,1024,256,4,6,0,0.001,50,4,yes,646.7671,10.78,1.346300,43,679.524536,0.42,322.90,1.0
1,a2_50,1024,256,4,6,0,0.001,50,4,yes,631.2694,10.52,1.177220,46,289.333679,0.36,281.15,1.0
2,a3_50,1024,256,4,6,0,0.001,50,4,yes,662.6886,11.04,1.390447,36,275.937103,0.40,321.41,1.0
3,b_50,1024,256,4,6,0,0.001,50,4,yes,652.5901,10.88,1.352880,36,442.333618,0.24,322.21,1.0
4,c_50,1024,256,4,6,0,0.001,50,4,yes,643.5783,10.73,1.369530,36,364.786407,0.32,321.04,1.0
5,d_50,1024,256,4,6,0,0.001,50,4,yes,634.2541,10.57,1.167735,50,486.180145,0.29,278.94,1.0
6,a1,1024,256,4,6,0,0.001,20,4,yes,253.2759,4.22,1.378125,20,94.867638,0.55,322.85,1.0
7,a2,1024,256,4,6,0,0.001,20,4,yes,262.6371,4.38,1.272294,20,103.435127,0.46,278.76,1.0
8,a3,1024,256,4,6,0,0.001,20,4,yes,263.4213,4.39,1.408199,20,202.802673,0.40,321.03,1.0
9,b,1024,256,4,6,0,0.001,20,4,yes,263.1353,4.39,1.351377,20,88.056297,0.58,321.76,1.0


In [5]:
model_df.to_excel(f"{PATH_MODELS}/comparison_model_stats.xlsx")